This notebook contains the Bias correction over precipitation with the method BcsdPrecipitation(return_anoms=False)

In [2]:
# import modules and functions

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# import data
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6
from Functions_ImportData import import_filtered_NOAA_obs
from Functions_ImportData import import_treat_obs_NOAA
from Functions_ImportData import import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA
from Bias_correction_function import BC
from Bias_correction_function import treat_data_for_test
from Bias_correction_function import BCSD_Precipitation_return_anoms_to_apply
from Functions_Indicators import add_year_month_season

In [3]:
climate_var = 'pr' # 'tas'
unit='mm_per_day'
resolution = 'day'

start_y = 2030
stop_y = 2065
# precipitation : 'pr'

name_station = 'BEIRA, MZ'
name_project = 'WTP_Mutua_EIB'

In [4]:
# import observation data
data_obs_NOAA_filtered=import_filtered_NOAA_obs()

In [8]:
# import historic modeled data
climate_var_NEX_GDDP_CMIP6_EmplacementStation=import_treat_modeled_NEX_GDDP_CMIP6_close_to_stationNOAA(climate_var, unit)

In [9]:
# import data to correct
climate_var_NEX_GDDP_CMIP6_Project=import_treat_modeled_NEX_GDDP_CMIP6(climate_var,'mm_per_day','day',1950,2100)

In [10]:
climate_var_NEX_GDDP_CMIP6_Project

,Name project,Experiment,Model,Latitude,Longitude,Date,Mean of the daily precipitation rate mm_per_day,Year,Month,Season
0,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,01-01-1950,0.657509,1950,Jan,Humid
1,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,02-01-1950,0.442182,1950,Jan,Humid
2,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,03-01-1950,0.031404,1950,Jan,Humid
3,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,04-01-1950,15.071664,1950,Jan,Humid
4,WTP_Mutua_EIB,historical,ACCESS-CM2,-19.375,34.625,05-01-1950,12.865736,1950,Jan,Humid
...,...,...,...,...,...,...,...,...,...,...
8963095,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,27-12-2100,0.286506,2100,Dec,Humid
8963096,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,28-12-2100,0.050367,2100,Dec,Humid
8963097,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,29-12-2100,0.000000,2100,Dec,Humid
8963098,Pemba_EIB,ssp370,TaiESM1,-12.875,40.625,30-12-2100,0.581391,2100,Dec,Humid


In [11]:
climate_var_NEX_GDDP_CMIP6_Project_to_correct=climate_var_NEX_GDDP_CMIP6_Project[climate_var_NEX_GDDP_CMIP6_Project['Year'].between(start_y,stop_y)]

In [12]:
list_model=list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Model']))#.remove('NESM3')
list_model.remove('NESM3')

In [37]:
df_pr_bc_corrected=pd.DataFrame()
for name_project in list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Name project'])):
    climate_var_NEX_GDDP_CMIP6_Project_temp=climate_var_NEX_GDDP_CMIP6_Project_to_correct[climate_var_NEX_GDDP_CMIP6_Project_to_correct['Name project']==name_project]
    for scenario in list(set(climate_var_NEX_GDDP_CMIP6_Project_to_correct['Experiment'])):
        climate_var_NEX_GDDP_CMIP6_Project_temp_2=climate_var_NEX_GDDP_CMIP6_Project_temp[climate_var_NEX_GDDP_CMIP6_Project_temp['Experiment']==scenario]
        for model in list_model:
            print('For '+name_project+', '+scenario+' and '+model)
            climate_var_NEX_GDDP_CMIP6_Project_temp_3=climate_var_NEX_GDDP_CMIP6_Project_temp_2[climate_var_NEX_GDDP_CMIP6_Project_temp_2['Model']==model]
            df_pr = treat_data_for_test(data_obs_NOAA_filtered,'PRCP',climate_var_NEX_GDDP_CMIP6_EmplacementStation,'Mean of the daily precipitation rate mm/day','BEIRA, MZ',model)
            (X_pcp,y_pcp,out)=BCSD_Precipitation_return_anoms_to_apply(df_pr,climate_var_NEX_GDDP_CMIP6_Project_temp_3[['Date','Mean of the daily precipitation rate mm_per_day']].dropna().reset_index(drop=True))
            out = out.reset_index()
            out['Name project']=name_project
            out['Experiment']=scenario
            out['Model']=model
            df_pr_bc_corrected=pd.concat([df_pr_bc_corrected,out])

For Chimoio_WTP_EIB, ssp126 and ACCESS-CM2
For Chimoio_WTP_EIB, ssp126 and TaiESM1
For Chimoio_WTP_EIB, ssp126 and NorESM2-MM
For Chimoio_WTP_EIB, ssp126 and CMCC-CM2-SR5
For Chimoio_WTP_EIB, ssp126 and MPI-ESM1-2-LR
For Chimoio_WTP_EIB, ssp126 and NorESM2-LM
For Chimoio_WTP_EIB, ssp126 and MPI-ESM1-2-HR
For Chimoio_WTP_EIB, ssp126 and MIROC6
For Chimoio_WTP_EIB, ssp126 and CanESM5
For Chimoio_WTP_EIB, ssp126 and ACCESS-ESM1-5
For Chimoio_WTP_EIB, ssp126 and BCC-CSM2-MR
For Chimoio_WTP_EIB, ssp126 and CMCC-ESM2
For Chimoio_WTP_EIB, ssp126 and IITM-ESM
For Chimoio_WTP_EIB, ssp126 and MRI-ESM2-0
For Chimoio_WTP_EIB, ssp370 and ACCESS-CM2
For Chimoio_WTP_EIB, ssp370 and TaiESM1
For Chimoio_WTP_EIB, ssp370 and NorESM2-MM
For Chimoio_WTP_EIB, ssp370 and CMCC-CM2-SR5
For Chimoio_WTP_EIB, ssp370 and MPI-ESM1-2-LR
For Chimoio_WTP_EIB, ssp370 and NorESM2-LM
For Chimoio_WTP_EIB, ssp370 and MPI-ESM1-2-HR
For Chimoio_WTP_EIB, ssp370 and MIROC6
For Chimoio_WTP_EIB, ssp370 and CanESM5
For Chimoio_WT

For Pemba_EIB, ssp585 and NorESM2-LM
For Pemba_EIB, ssp585 and MPI-ESM1-2-HR
For Pemba_EIB, ssp585 and MIROC6
For Pemba_EIB, ssp585 and CanESM5
For Pemba_EIB, ssp585 and ACCESS-ESM1-5
For Pemba_EIB, ssp585 and BCC-CSM2-MR
For Pemba_EIB, ssp585 and CMCC-ESM2
For Pemba_EIB, ssp585 and IITM-ESM
For Pemba_EIB, ssp585 and MRI-ESM2-0
For Pemba_EIB, ssp245 and ACCESS-CM2
For Pemba_EIB, ssp245 and TaiESM1
For Pemba_EIB, ssp245 and NorESM2-MM
For Pemba_EIB, ssp245 and CMCC-CM2-SR5
For Pemba_EIB, ssp245 and MPI-ESM1-2-LR
For Pemba_EIB, ssp245 and NorESM2-LM
For Pemba_EIB, ssp245 and MPI-ESM1-2-HR
For Pemba_EIB, ssp245 and MIROC6
For Pemba_EIB, ssp245 and CanESM5
For Pemba_EIB, ssp245 and ACCESS-ESM1-5
For Pemba_EIB, ssp245 and BCC-CSM2-MR
For Pemba_EIB, ssp245 and CMCC-ESM2
For Pemba_EIB, ssp245 and IITM-ESM
For Pemba_EIB, ssp245 and MRI-ESM2-0


In [14]:
df_pr_bc_corrected=df_pr_bc_corrected.reset_index(drop=True)

In [34]:
df_t=pd.to_datetime(df_pr_bc_corrected[['Date']].Date,format='%Y-%d-%m')

In [36]:
df_pr_bc_corrected[['Date']]

,Date
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
2942775,NaN
2942776,NaN
2942777,NaN
2942778,NaN


In [35]:
df_t[0]

NaT

In [ ]:
#df_pr_bc_corrected[['Date']].iloc[1000][0].strftime('%Y-%m-%d')

In [ ]:
#df_pr_bc_corrected[['Date']].iloc[1000][0]

In [ ]:
#len(df_pr_bc_corrected[['Date']])

In [ ]:
# convert date in str format

In [ ]:
# teste .date()

In [15]:
Time = df_pr_bc_corrected[['Date']]

In [16]:
df_pr_bc_corrected[['Date']]= np.nan

In [25]:
df_pr_bc_corrected.loc[5,'Date']=Time.loc[6,'Date']#.date().strftime('%Y-%m-%d')

In [18]:
df_pr_bc_corrected.loc[5,'Date']

'2030-01-06'

In [19]:
column_date = 'Date'
df = df_pr_bc_corrected#.reset_index()

In [20]:
Year = df[[column_date]].values.reshape(len(df[[column_date]].values),)

In [21]:
Year[5][0:4]

'2030'

In [ ]:
for date in np.arange(0,len(df_pr_bc_corrected[['Date']])):
    df_pr_bc_corrected.loc[date,'Date']=Time.loc[date,'Date'].date().strftime('%Y-%m-%d')

In [ ]:
df_pr_bc_corrected[['Date']].reset_index().loc[0]['Date']

In [ ]:
df_pr_bc_corrected[['Date']].reset_index(drop=True)

In [ ]:
df_pr_bc_corrected = add_year_month_season(df_pr_bc_corrected.reset_index(),'Date')

In [1]:
column_date = 'Date'
df = df_pr_bc_corrected#.reset_index()

NameError: name 'df_pr_bc_corrected' is not defined

In [ ]:
Year = df[[column_date]].values.reshape(len(df[[column_date]].values),)
Month = df[[column_date]].values.reshape(len(df[[column_date]].values),)
Season = df[[column_date]].values.reshape(len(df[[column_date]].values),)

if str(Year[1]).find('-')==2 or str(Year[1]).find('/')==2:
    for i in np.arange(0,len(df[[column_date]].values)):
        Year[i]=int(Year[i][6:10])
        Month[i]=int(Month[i][3:5])
        if Month[i]>3 and Month[i]<10: # dry season in Mozambique is between April and September
            Season[i]='Dry'
        else:# humid season is between October and March
            Season[i]='Humid'

        Month[i]=str_month(Month[i])

if str(Year[1]).find('-')==4 or str(Year[1]).find('/')==4:
    for i in np.arange(0,len(df[[column_date]].values)):
        Year[i]=int(Year[i][0:4])
        Month[i]=int(Month[i][5:7])
        if Month[i]>3 and Month[i]<10: # dry season in Mozambique is between April and September
            Season[i]='Dry'
        else:# humid season is between October and March
            Season[i]='Humid'

        Month[i]=str_month(Month[i])

df['Year'] = Year
df['Month'] = Month
df['Season'] = Season

In [ ]:
Year

In [ ]:
df_pr_bc_corrected

In [ ]:
path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6-AllMoz\csv_file'

In [ ]:
df_pr_bc_corrected.to_csv(os.path.join(path,climate_var,climate_var+'_'+unit+'_'+resolution+'_'+str(start_y)+'-'+str(stop_y)+'_BiasCorrected',climate_var+'_'+'mm_per_day'+resolution+str(start_y)+'-'+str(stop_y)+'_BiasCorrected.csv'))